In [5]:
import pandas as pd
from datetime import datetime
from ast import literal_eval
import tensorflow as tf
import numpy as np
import tensorflow_recommenders as tfrs
from typing import Dict, Text



In [6]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
movies = pd.read_csv('movies_metadata.csv').\
                     drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).\
                     drop([19730, 29503, 35587]) # Incorrect data type

movies['id'] = movies['id'].astype('int64')

df = movies.merge(keywords, on='id').\
    merge(credits, on='id')

df['original_language'] = df['original_language'].fillna('')
df['runtime'] = df['runtime'].fillna(0)
df['tagline'] = df['tagline'].fillna('')

df.dropna(inplace=True)

C:\Users\acer\AppData\Local\Temp\ipykernel_30396\1889031945.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_metadata.csv').\


In [7]:
def get_text(text, obj='name'):
    text = literal_eval(text)
    
    if len(text) == 1:
        for i in text:
            return i[obj]
    else:
        s = []
        for i in text:
            s.append(i[obj])
        return ', '.join(s)
    
df['genres'] = df['genres'].apply(get_text)
df['production_companies'] = df['production_companies'].apply(get_text)
df['production_countries'] = df['production_countries'].apply(get_text)
df['crew'] = df['crew'].apply(get_text)
df['spoken_languages'] = df['spoken_languages'].apply(get_text)
df['keywords'] = df['keywords'].apply(get_text)

# New columns
df['characters'] = df['cast'].apply(get_text, obj='character')
df['actors'] = df['cast'].apply(get_text)

df.drop('cast', axis=1, inplace=True)
df = df[~df['original_title'].duplicated()]
df = df.reset_index(drop=True)

In [8]:
df.head()

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,...,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,crew,characters,actors
0,False,30000000,"Animation, Comedy, Family",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,...,373554033.0,81.0,English,,7.7,5415.0,"jealousy, toy, boy, friendship, friends, rival...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...","Woody (voice), Buzz Lightyear (voice), Mr. Pot...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,False,65000000,"Adventure, Fantasy, Family",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,...,262797249.0,104.0,"English, Français",Roll the dice and unleash the excitement!,6.9,2413.0,"board game, disappearance, based on children's...","Larry J. Franco, Jonathan Hensleigh, James Hor...","Alan Parrish, Samuel Alan Parrish / Van Pelt, ...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,False,0,"Romance, Comedy",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros., Lancaster Gate",United States of America,...,0.0,101.0,English,Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,"fishing, best friend, duringcreditsstinger, ol...","Howard Deutch, Mark Steven Johnson, Mark Steve...","Max Goldman, John Gustafson, Ariel Gustafson, ...","Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,False,16000000,"Comedy, Drama, Romance",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,...,81452156.0,127.0,English,Friends are the people who let you be yourself...,6.1,34.0,"based on novel, interracial relationship, sing...","Forest Whitaker, Ronald Bass, Ronald Bass, Ezr...","Savannah 'Vannah' Jackson, Bernadine 'Bernie' ...","Whitney Houston, Angela Bassett, Loretta Devin..."
4,False,0,Comedy,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions, Touchstone Pictures",United States of America,...,76578911.0,106.0,English,Just When His World Is Back To Normal... He's ...,5.7,173.0,"baby, midlife crisis, confidence, aging, daugh...","Alan Silvestri, Elliot Davis, Nancy Meyers, Na...","George Banks, Nina Banks, Franck Eggelhoffer, ...","Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [9]:
ratings_df = pd.read_csv('ratings_small.csv')

ratings_df['date'] = ratings_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
ratings_df.drop('timestamp', axis=1, inplace=True)

ratings_df = ratings_df.merge(df[['id', 'original_title', 'genres', 'overview']], left_on='movieId',right_on='id', how='left')
ratings_df = ratings_df[~ratings_df['id'].isna()]
ratings_df.drop('id', axis=1, inplace=True)
ratings_df.reset_index(drop=True, inplace=True)

ratings_df.head()

,userId,movieId,rating,date,original_title,genres,overview
0,1,1371,2.5,2009-12-14 03:52:15,Rocky III,Drama,"Now the world champion, Rocky Balboa is living..."
1,1,1405,1.0,2009-12-14 03:53:23,Greed,"Drama, History",Greed is the classic 1924 silent film by Erich...
2,1,2105,4.0,2009-12-14 03:52:19,American Pie,"Comedy, Romance","At a high-school party, four friends find that..."
3,1,2193,2.0,2009-12-14 03:53:18,My Tutor,"Comedy, Drama, Romance",High school senior Bobby Chrystal fails his Fr...
4,1,2294,2.0,2009-12-14 03:51:48,Jay and Silent Bob Strike Back,Comedy,When Jay and Silent Bob learn that their comic...


In [10]:
movies_df = df[['id', 'original_title']]
movies_df.rename(columns={'id':'movieId'}, inplace=True)
movies_df.head()

C:\Users\acer\AppData\Local\Temp\ipykernel_30396\1980486665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.rename(columns={'id':'movieId'}, inplace=True)


,movieId,original_title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [11]:
ratings_df['userId'] = ratings_df['userId'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df[['userId', 'original_title', 'rating']]))
movies = tf.data.Dataset.from_tensor_slices(dict(movies_df[['original_title']]))

ratings = ratings.map(lambda x: {
    "original_title": x["original_title"],
    "userId": x["userId"],
    "rating": float(x["rating"])
})

movies = movies.map(lambda x: x["original_title"])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [12]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = ratings.take(35_000)
test = ratings.skip(35_000).take(8_188)

Total Data: 43188


In [13]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["userId"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Movies: {}'.format(len(unique_movie_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Movies: 42373
Unique users: 671


In [14]:
class MovieModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["userId"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["original_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [15]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
35/35 [==============================] - 78s 2s/step - root_mean_squared_error: 1.4681 - factorized_top_k/top_1_categorical_accuracy: 2.8571e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0079 - factorized_top_k/top_10_categorical_accuracy: 0.0184 - factorized_top_k/top_50_categorical_accuracy: 0.1035 - factorized_top_k/top_100_categorical_accuracy: 0.1733 - loss: 6814.1162 - regularization_loss: 0.0000e+00 - total_loss: 6814.1162
Epoch 2/3
35/35 [==============================] - 78s 2s/step - root_mean_squared_error: 1.0140 - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0195 - factorized_top_k/top_10_categorical_accuracy: 0.0450 - factorized_top_k/top_50_categorical_accuracy: 0.2044 - factorized_top_k/top_100_categorical_accuracy: 0.3231 - loss: 6454.3204 - regularization_loss: 0.0000e+00 - total_loss: 6454.3204
Epoch 3/3
35/35 [==============================] - 77s 2s/step - root_mean_squared_error: 1.0176 - fac

In [16]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

9/9 [==============================] - 17s 2s/step - root_mean_squared_error: 1.1592 - factorized_top_k/top_1_categorical_accuracy: 7.3278e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0045 - factorized_top_k/top_10_categorical_accuracy: 0.0096 - factorized_top_k/top_50_categorical_accuracy: 0.0393 - factorized_top_k/top_100_categorical_accuracy: 0.0738 - loss: 5730.6018 - regularization_loss: 0.0000e+00 - total_loss: 5730.6018

Retrieval top-100 accuracy: 0.074
Ranking RMSE: 1.159


In [17]:
def predict_movie(user, top_n=5):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

def predict_rating(user, movie) -> float:
    trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
          "userId": np.array([str(user)]),
          "original_title": np.array([movie])
      })
    print("Predicted rating for {}: {}".format(movie, predicted_rating.numpy()[0][0]))
    return (predicted_rating.numpy()[0][0])

In [27]:
predict_movie(55, 5)

Top 5 recommendations for user 55:

1. Dont Look Back
2. L'Aile ou la Cuisse
3. La vita è bella
4. Varjoja paratiisissa
5. Shaft in Africa


In [19]:
predict_rating(55, 'Himenóptero')

Predicted rating for Himenóptero: 3.6730897426605225


3.6730897

In [20]:
ratings_df[ratings_df['userId'] == '55']

,userId,movieId,rating,date,original_title,genres,overview
3431,55,5,3.0,1997-02-14 14:29:48,Four Rooms,"Crime, Comedy",It's Ted the Bellhop's first night on the job....
3432,55,6,5.0,1997-02-14 14:29:48,Judgment Night,"Action, Thriller, Crime","While racing to a boxing match, Frank, Mike, J..."
3433,55,65,5.0,1997-02-14 14:35:15,8 Mile,Drama,The setting is Detroit in 1995. The city is di...
3434,55,74,3.0,1997-02-14 14:37:18,War of the Worlds,"Adventure, Thriller, Science Fiction",Ray Ferrier is a divorced dockworker and less-...
3435,55,79,3.0,1997-02-14 14:32:52,英雄,"Drama, Adventure, Action, History",One man defeated three assassins who sought to...
3436,55,95,3.0,1997-02-14 14:29:01,Armageddon,"Action, Thriller, Science Fiction, Adventure",When an asteroid threatens to collide with Ear...
3437,55,100,3.0,1997-02-14 14:37:17,"Lock, Stock and Two Smoking Barrels","Comedy, Crime",A card sharp and his unwillingly-enlisted frie...
3438,55,104,5.0,1997-02-14 14:32:11,Lola rennt,"Action, Drama, Thriller",Lola receives a phone call from her boyfriend ...
3439,55,112,3.0,1997-02-14 14:32:11,Italiensk for begyndere,"Comedy, Drama, Romance",This fifth Danish Dogme film is about six vuln...
3440,55,141,4.0,1997-02-14 14:29:01,Donnie Darko,"Fantasy, Drama, Mystery","After narrowly escaping a bizarre accident, a ..."


In [24]:
# Get meta data for predicted movie
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(['55']))
pred_movies = pd.DataFrame({'original_title': [i.decode('utf-8') for i in titles[0,:10].numpy()]})

pred_df = pred_movies.merge(ratings_df[['original_title', 'genres', 'overview']], on='original_title', how='left')
pred_df = pred_df[~pred_df['original_title'].duplicated()]
pred_df.reset_index(drop=True, inplace=True)
pred_df.index = np.arange(1, len(pred_df)+1)

pred_df

,original_title,genres,overview
1,Dont Look Back,"Documentary, Music",In this wildly entertaining vision of one of t...
2,L'Aile ou la Cuisse,Comedy,"Charles Duchemin, a well-known gourmet and pub..."
3,La vita è bella,"Comedy, Drama",A touching story of an Italian book seller of ...
4,Varjoja paratiisissa,"Drama, Comedy","An episode in the life of Nikander, a garbage ..."
5,Shaft in Africa,"Adventure, Action, Thriller, Crime, Mystery",Detective John Shaft travels incognito to Ethi...
6,Poltergeist,Horror,"Steve Freeling lives with his wife, Diane, and..."
7,Monty Python and the Holy Grail,"Adventure, Comedy, Fantasy","King Arthur, accompanied by his squire, recrui..."
8,"Nosferatu, eine Symphonie des Grauens","Fantasy, Horror",Vampire Count Orlok is interested in a new res...
9,Evil Dead II,"Horror, Comedy, Fantasy",Ash Williams and his girlfriend Linda find a l...
10,A Hard Day's Night,"Comedy, Music","Capturing John Lennon, Paul McCartney, George ..."


As we can observe user with id == 55 has a very wide range of favourite categories, these are the top 5 movies recommended for him ^

In [22]:
error = 0.0
num_of_records = 0
for _, movie in ratings_df[ratings_df['userId'] == '55'].iterrows():
    val = predict_rating(55, movie[4])
    print("expected: {}".format(movie[2]))
    error += abs(val - movie[2])
    num_of_records += 1
    
print("Average error in Deep Learning while predicting movie rating is ", error/num_of_records)

Predicted rating for Four Rooms: 3.2717835903167725
expected: 3.0
Predicted rating for Judgment Night: 3.9165825843811035
expected: 5.0
Predicted rating for 8 Mile: 3.18035626411438
expected: 5.0
Predicted rating for War of the Worlds: 3.35383939743042
expected: 3.0
Predicted rating for 英雄: 3.484426498413086
expected: 3.0
Predicted rating for Armageddon: 3.6071925163269043
expected: 3.0
Predicted rating for Lock, Stock and Two Smoking Barrels: 3.908457040786743
expected: 3.0
Predicted rating for Lola rennt: 3.5983543395996094
expected: 5.0
Predicted rating for Italiensk for begyndere: 3.403775691986084
expected: 3.0
Predicted rating for Donnie Darko: 3.8838467597961426
expected: 4.0
Predicted rating for The 39 Steps: 4.365431308746338
expected: 5.0
Predicted rating for Shaft in Africa: 3.498244047164917
expected: 3.0
Predicted rating for Men in Black II: 4.325704097747803
expected: 3.0
Predicted rating for Poltergeist: 3.2877635955810547
expected: 3.0
Predicted rating for La vita è bel

In [26]:
dl_predict_movie_start = datetime.now()
for i in range(30): #measure performance for predicting 5 movies for 5 users 
    predict_movie(i, 5)
dl_predict_movie_end = datetime.now()
print(dl_predict_movie_end - dl_predict_movie_start)

Top 5 recommendations for user 0:

1. Les Vacances de Monsieur Hulot
2. Die Nibelungen, Teil 1: Siegfried
3. A Dandy in Aspic
4. Mere Brother Ki Dulhan
5. True Romance
Top 5 recommendations for user 1:

1. Vivement dimanche!
2. Jack & Sarah
3. Rocky III
4. Sleepy Hollow
5. El otro lado de la cama
Top 5 recommendations for user 2:

1. Cat on a Hot Tin Roof
2. My Own Private Idaho
3. The Green Mile
4. Berlin: Die Sinfonie der Grosstadt
5. Batman Begins
Top 5 recommendations for user 3:

1. Once Were Warriors
2. The Million Dollar Hotel
3. Terminator 3: Rise of the Machines
4. Солярис
5. Confession of a Child of the Century
Top 5 recommendations for user 4:

1. Infinity
2. Carla's Song
3. James Dean
4. Kiss of the Dragon
5. Roustabout
Top 5 recommendations for user 5:

1. New York Doll
2. リング
3. Star 80
4. Martha – Meet Frank, Daniel and Laurence
5. Ober
Top 5 recommendations for user 6:

1. Don't Make Waves
2. Secret Agent
3. Cyberjack
4. Read It and Weep
5. Speed 2: Cruise Control
Top 5